In [1]:
import pyspark
import pandas as pd
import numpy as np
from pydataset import data as pydata
from vega_datasets import data
from pyspark.sql import functions as F

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
pd_df = pd.DataFrame(['c++','python','cobol','c#'],
                    columns=(['language']))
pd_df

,language
0,c++
1,python
2,cobol
3,c#


In [4]:
sp_df = spark.createDataFrame(pd_df)
sp_df.show()

+--------+
|language|
+--------+
|     c++|
|  python|
|   cobol|
|      c#|
+--------+



In [5]:
sp_df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|       4|
|   mean|    null|
| stddev|    null|
|    min|      c#|
|    max|  python|
+-------+--------+



In [6]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
sp_df.count()

4

In [8]:
mpg = spark.createDataFrame(pydata('mpg'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
message1 = F.concat(F.lit('The '), F.col('year'), F.lit(' '),
F.col('manufacturer'), F.lit(' '), F.col('model'), F.lit(' '),
F.lit('has a '), F.col('cyl'), F.lit(" cylinder engine."))

In [10]:
mpg.select(message1.alias('message')).show(5, truncate=False)

+-----------------------------------------+
|message                                  |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



In [11]:
mpg.select(F.substring_index('trans', '(', 1).alias('tran')).show(5)

+------+
|  tran|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
+------+
only showing top 5 rows



In [14]:
tips = spark.createDataFrame(pydata('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [15]:
tips.where(tips.smoker =='Yes').count() / tips.count()

0.38114754098360654

In [16]:
tip_percent = F.round((tips.tip / tips.total_bill), 4)\
                .alias('tip_percent')
tips.select('sex', 'smoker', tip_percent).show(5)

+------+------+-----------+
|   sex|smoker|tip_percent|
+------+------+-----------+
|Female|    No|     0.0594|
|  Male|    No|     0.1605|
|  Male|    No|     0.1666|
|  Male|    No|     0.1398|
|Female|    No|     0.1468|
+------+------+-----------+
only showing top 5 rows



In [17]:
tips.groupBy('sex','smoker').agg(F.round(F.avg(tip_percent), 4)
                    .alias('tip_percent')).show(5)

+------+------+-----------+
|   sex|smoker|tip_percent|
+------+------+-----------+
|  Male|    No|     0.1607|
|  Male|   Yes|     0.1528|
|Female|    No|     0.1569|
|Female|   Yes|     0.1821|
+------+------+-----------+



In [18]:
weather = data.seattle_weather().assign(date=
                                lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [19]:
C_max = (weather.temp_max * 9/5) + 32
C_min = (weather.temp_min * 9/5) + 32
weather.select(C_max.alias('C_max') , C_min.alias('C_min')).show(5)

+-----+-----+
|C_max|C_min|
+-----+-----+
|55.04| 41.0|
|51.08|37.04|
|53.06|44.96|
|53.96|42.08|
|48.02|37.04|
+-----+-----+
only showing top 5 rows



In [20]:
weather.withColumn('month', F.month('date')).groupBy('month')\
        .agg(F.avg('precipitation').alias('avg_rain'))\
        .sort('avg_rain', ascending=False).show()

+-----+-------------------+
|month|           avg_rain|
+-----+-------------------+
|   11|  5.354166666666667|
|   12|  5.021774193548388|
|    3|  4.888709677419355|
|   10|  4.059677419354839|
|    1| 3.7580645161290316|
|    2|  3.734513274336283|
|    4|  3.128333333333333|
|    9| 1.9624999999999997|
|    5| 1.6733870967741935|
|    8| 1.3201612903225806|
|    6| 1.1075000000000002|
|    7|0.38870967741935486|
+-----+-------------------+



In [21]:
weather.withColumn('year', F.year('date')).groupBy('year')\
        .agg(F.avg('wind').alias('avg_wind'))\
        .sort('avg_wind', ascending=False).show()

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2012| 3.400819672131147|
|2014|3.3876712328767136|
|2015|  3.15972602739726|
|2013|3.0158904109589044|
+----+------------------+



In [22]:
weather.where(F.month('date') == 1).select('weather')\
        .groupBy('weather').count().sort('count', ascending=False)\
        .show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [23]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [24]:
q3_2015_rainy_days = \
            weather.where((F.quarter('date') == 3) & \
                          (F.year('date') == 2015))\
                   .where(weather.weather =='rain').count()
q3_2015_days = weather.where((F.month('date').isin([7,8,9])) & \
                             (F.year('date') == 2015)).count()
q3_2015_rainy_days / q3_2015_days

0.021739130434782608

In [25]:
weather.where(weather.precipitation > 0).groupBy(F.year('date'))\
            .count().sort('year(date)').show()

+----------+-----+
|year(date)|count|
+----------+-----+
|      2012|  177|
|      2013|  152|
|      2014|  150|
|      2015|  144|
+----------+-----+



In [26]:
# rainy = weather.when()

In [27]:
# days_rained = weather.where(weather.precipitation > 0)\
#                     .groupBy(F.year('date')).count()
# weather.select(days_rained)